In [68]:
from keras import models, layers, Input

input_tensor = Input(shape=(64,))

x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = models.Model(input_tensor, output_tensor)
model.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_32 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_33 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_34 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [70]:
import numpy as np

train_x = np.random.random((1000, 64))
train_y = np.random.random((1000, 10))

model.fit(train_x, train_y, epochs=10, batch_size=128)
score = model.evaluate(train_x, train_y)

Epoch 1/10
8/8 [==============================] - 0s 2ms/step - loss: 12.0036 - acc: 0.0977
Epoch 2/10
8/8 [==============================] - 0s 4ms/step - loss: 12.5776 - acc: 0.0799
Epoch 3/10
8/8 [==============================] - 0s 4ms/step - loss: 13.3381 - acc: 0.0800
Epoch 4/10
8/8 [==============================] - 0s 3ms/step - loss: 14.7565 - acc: 0.0854
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 16.4107 - acc: 0.0763
Epoch 6/10
8/8 [==============================] - 0s 3ms/step - loss: 18.7997 - acc: 0.0856
Epoch 7/10
8/8 [==============================] - 0s 12ms/step - loss: 21.5692 - acc: 0.0843
Epoch 8/10
8/8 [==============================] - 0s 5ms/step - loss: 25.0705 - acc: 0.0834
Epoch 9/10
8/8 [==============================] - 0s 3ms/step - loss: 28.4168 - acc: 0.0749
Epoch 10/10
32/32 [==============================] - 0s 988us/step - loss: 35.3195 - acc: 0.0860


In [78]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, 1, size=(num_samples, answer_vocabulary_size))

model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 4s 103ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 1s 88ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 1s 88ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 1s 87ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 1s 89ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 1s 88ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 1s 88ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 8/10
8/8 [==============================] - 1s 90ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 9/10
8/8 [==============================] - 1s 88ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 10/10
8/8 [==============================] - 1s 105ms/step - loss: 0.0000e+00 - acc: 0.0000e+

In [81]:
vocabulary_size = 50000
num_income_group = 10

posts_input = Input(shape=(None,), dtype='int32', name='post')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age = layers.Dense(1, name='age')(x)
income = layers.Dense(num_income_group, activation='softmax', name='income')(x)
gender = layers.Dense(1, name='gender', activation='sigmoid')(x)
model = models.Model(posts_input, [age, income, gender])

In [82]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'], loss_weights=[0.25, 1., 10.])